In [1]:
import pandas as pd
import pulp

# Using PuLP to Optimize Lineups

## Loading Player Data

In [2]:
df = pd.read_csv('data/pool.csv')

In [3]:
df.head()

,first_name,last_name,pos,team,salary,fppg,full_name
0,Buffalo,Defense,DST,BUF,3200,21.0,Buffalo_Defense
1,LA Rams,Defense,DST,LAR,3300,20.0,LA Rams_Defense
2,New Orleans,Defense,DST,NOR,3400,17.0,New Orleans_Defense
3,Baltimore,Defense,DST,BAL,4200,11.0,Baltimore_Defense
4,Denver,Defense,DST,DEN,2400,10.0,Denver_Defense


## Setting up the PuLP problem

In [4]:
# setup problem
prob = pulp.LpProblem('FPOpt', pulp.LpMaximize)

# create variables
# key is Player_{full_name}, value is pulp.LpVariable
player_vars = pulp.LpVariable.dicts('Player', df.full_name, cat='Binary')


In [5]:
# take a look at dict item
# key is str, value is LpVariable
k = list(player_vars.keys())[0]
print(type(k))
print(type(player_vars[k]))


<class 'str'>
<class 'pulp.pulp.LpVariable'>


## Adding the Objective Function

In [6]:
# objective function - sum of fantasy points
prob += pulp.lpSum([pts * pvar for pvar, pts in zip(player_vars.values(), df.fppg)])

## Core Constraints

In [7]:
# constraint: salary must not exceed cap
prob += pulp.lpSum([sal * pvar for pvar, sal in zip(player_vars.values(), df.salary)]) <= 50000

In [8]:
# constraint: lineup must have 9 players
prob += pulp.lpSum(player_vars.values()) == 9

In [9]:
# add positional constraints
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'QB']) == 1
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'RB']) >= 2
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'RB']) <= 3
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'WR']) >= 3
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'WR']) <= 4
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'TE']) >= 1
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'TE']) <= 2
prob += pulp.lpSum([v for v, pos in zip(player_vars.values(), df.pos) 
                    if pos == 'DST']) == 1

## Optional Constraints

In [26]:
# stacking QB and receiver
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
        df.loc[df.full_name == k, 'team'].values[0] == row.team and
                  df.loc[df.full_name == k, 'pos'].values[0] == 'WR'] +
                  [-1 * player_vars[row.full_name]]) >= 0  

In [11]:
# stacking QB and two other players
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
                        df.loc[df.full_name == k, 'team'].values[0] == row.team and
                        df.loc[df.full_name == k, 'pos'].values[0] in ('WR', 'TE', 'RB')] +
                        [-2 * player_vars[row.full_name]]) >= 0  

In [ ]:
# include a bringback
for row in df.loc[df.pos == 'QB', :].itertuples(index=False):
    prob += pulp.lpSum([v for k,v in player_vars.items() if
        df.loc[df.full_name == k, 'team'].values[0] == row.opp and
                  df.loc[df.full_name == k, 'pos'].values[0] == 'WR'] +
                  [-1 * player_vars[row.full_name]]) >= 0

## Solve Problem

In [13]:
# solve problem and print solution
prob.solve()
chosen = [v.name for v in prob.variables() if v.varValue == 1]
solution = df.loc[df.full_name.isin([c.replace('Player_', '') for c in chosen]), list(df.columns)[0:-1]]
print(solution)
print(f'\n{solution.fppg.sum()}')

    first_name last_name  pos team  salary   fppg
0      Buffalo   Defense  DST  BUF    3200  21.00
14         Tom     Brady   QB  TAM    7200  34.26
42    Jonathan    Taylor   RB  IND    7400  41.40
45      Darwin  Thompson   RB  KAN    4400  30.00
87       Chris   Herndon   TE  NYJ    2800  19.30
110    Brandin     Cooks   WR  HOU    6900  42.60
111     Marvin     Jones   WR  DET    5100  41.00
112    Antonio     Brown   WR  TAM    5500  39.80
115      Chris    Godwin   WR  TAM    6600  33.30

302.66
